# 🎮 Battle City RL - DreamerV3 Training

**Setup:** `Runtime` > `Change runtime type` > **T4 GPU**

### 1. Clone Repository

In [ ]:
import os

# Clone if not exists
if not os.path.exists('/content/battle_city'):
    !git clone https://github.com/Nikolay1221/x.git /content/battle_city
else:
    print('Repo already cloned!')

# Change directory
os.chdir('/content/battle_city')
print(f'Working dir: {os.getcwd()}')
!ls -la

### 2. Install System Dependencies

In [ ]:
!apt-get update -qq
!apt-get install -y -qq freeglut3-dev xvfb
print('✅ System deps installed')

### 3. Install Python Packages

In [ ]:
# Base packages
!pip install -q gym gymnasium opencv-python matplotlib numpy
!pip install -q nes-py
print('✅ Base packages')

In [ ]:
# DreamerV3 from GitHub (official)
!pip install -q git+https://github.com/danijar/embodied.git
!pip install -q git+https://github.com/danijar/dreamerv3.git
print('✅ DreamerV3')

In [ ]:
# JAX with CUDA (pinned compatible versions)
!pip install -q jax==0.4.30 jaxlib==0.4.30+cuda12.cudnn89 -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
print('✅ JAX (GPU)')

### 4. Verify Setup

In [ ]:
import os
os.chdir('/content/battle_city')  # Ensure correct dir

required = ['BattleCity.nes', 'battle_city_env.py', 'dreamer_wrapper.py', 'train_dreamer.py']
missing = [f for f in required if not os.path.exists(f)]

if missing:
    print(f'❌ Missing: {missing}')
else:
    print('✅ All files present!')

# Test JAX
try:
    import jax
    print(f'✅ JAX version: {jax.__version__}')
    print(f'✅ JAX devices: {jax.devices()}')
except Exception as e:
    print(f'❌ JAX: {e}')

# Test DreamerV3
try:
    import dreamerv3
    print('✅ DreamerV3 OK')
except Exception as e:
    print(f'❌ DreamerV3: {e}')

### 5. Start Training 🚀

In [ ]:
# Virtual display for NES emulator
!Xvfb :1 -screen 0 1024x768x24 &
import os
os.environ['DISPLAY'] = ':1'

# Ensure we're in project dir
os.chdir('/content/battle_city')

# Launch!
!python train_dreamer.py